In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_gamma = pd.read_csv('gamma-ray.csv')

display(df_gamma)

,seconds,count
0,116.0,0.0
1,112.0,0.0
2,160.0,0.0
3,51.5,0.0
4,102.0,1.0
...,...,...
95,38.7,1.0
96,41.8,0.0
97,81.7,0.0
98,88.3,0.0


In [6]:
# Model: G ~ Poisson(lambda * t)
# H0 = lamba1 = lambda2 = ... = lambda99
# HA = lambai not equal to lambdaj for some i and j

# What is(are) the most plausible parameter value(s) for the null model given the observations? Derive the MLE(s), i.e., maximum likelihood estimators. Calculate the value of the estimator(s) from the data.

total_counts = df_gamma['count'].sum()
total_time = df_gamma['seconds'].sum()
lambda_mle = total_counts / total_time

print(f"MLE for lambda: {lambda_mle:.3g}")

MLE for lambda: 0.00388


In [ ]:
# Determine the rejection region at a significance level of 0.05. 
from scipy.stats import chi2

df = 99
alpha = 0.05

critical_value = chi2.ppf(1 - alpha, df)
print(f"Critical value at 0.05: {critical_value:.3g}")

Critical value at 0.05: 123


In [ ]:
# Given the observed data, the value of the test statistic is: 

# Data already loaded as df with columns "counts" and "time"
counts = df_gamma["count"].to_numpy()
times = df_gamma["seconds"].to_numpy()

# Null MLE (common lambda)
lambda_hat = counts.sum() / times.sum()

# Alternative MLEs (interval-specific lambdas)
lambda_hats_alt = counts / times
lambda_hats_alt = np.where(times > 0, lambda_hats_alt, 0)

# Deviance (likelihood ratio statistic)
# Formula: 2 * sum[ G_i * log(G_i/(lambda_hat*t_i)) + lambda_hat*t_i - G_i ]
# Convention: 0 * log(0/μ) = 0 when G_i=0
mu_null = lambda_hat * times
terms = np.zeros_like(counts, dtype=float)

for i, (g, mu) in enumerate(zip(counts, mu_null)):
    if g == 0:
        terms[i] = mu  # since g*log(g/mu)=0, then term = mu - 0
    else:
        terms[i] = g * np.log(g / mu) + mu - g

test_statistic = 2 * terms.sum()

# Critical value at alpha=0.05, df=98
df_chi = len(counts) - 1
alpha = 0.05
critical_value = chi2.ppf(1 - alpha, df_chi)

# p-value
p_value = 1 - chi2.cdf(test_statistic, df_chi)

(test_statistic, critical_value, p_value)

## Problem 1.4 - Gene Expression



In [ ]:
# The data set golub consists of the expression levels of 3051 genes for 38 tumor mRNA samples. Each tumor mRNA sample comes from one patient (i.e. 38 patients total), and 27 of these tumor samples correspond to acute lymphoblastic leukemia (ALL) and the remaining 11 to acute myeloid leukemia (AML).

# You will need to discover how many genes can be used to differentiate the tumor types (meaning that their expression level differs between the two tumor types) using the uncorrected p-values, the Holm-Bonferroni correction, and (iii) the Benjamini-Hochberg correction?

In [14]:
import zipfile

with zipfile.ZipFile("release_statsreview_release1.zip") as zip_file:
    golub_data, golub_classnames = ( np.genfromtxt(zip_file.open('data_and_materials/golub_data/{}'.format(fname)), delimiter=',', names=True, converters={0: lambda s: int(s.strip(b'"'))}) for fname in ['golub.csv', 'golub_cl.csv'] )